In [ ]:
import numpy as np
import pytest

In [ ]:
from dabest._stats_tools import effsize
from dabest._stats_tools import confint_2group_diff as ci2g
from dabest import Dabest, PermutationTest
from data.mocked_data_test_08 import df_mini_meta, rep1_yes, rep1_no, rep2_yes, rep2_no, N, dabest_default_kwargs

Pre-compiling numba functions for DABEST...


Compiling numba functions: 100%|███████████████████████████████████████████████████████| 11/11 [00:22<00:00,  2.01s/it]

Numba compilation complete!


In [ ]:
unpaired = Dabest(data = df_mini_meta, idx =(("Rep1_No", "Rep1_Yes"), 
                                             ("Rep2_No", "Rep2_Yes")), 
                                             mini_meta=True,
                                             **dabest_default_kwargs)



In [ ]:
unpaired.mean_diff.mini_meta.bootstraps_var

array([ 1.51539707, 10.22387374])

test_mean_diff

In [ ]:
mean_diff = unpaired.mean_diff.results['difference'].to_list()
np_result = [np.mean(rep1_yes)-np.mean(rep1_no), 
             np.mean(rep2_yes)-np.mean(rep2_no)]
assert mean_diff == pytest.approx(np_result)

test_pooled_variances

In [ ]:
mini_meta_delta = unpaired.mean_diff.mini_meta

control_var    = mini_meta_delta.control_var
np_control_var = [np.var(rep1_no, ddof=1),
                  np.var(rep2_no, ddof=1)]
assert control_var == pytest.approx(np_control_var)

test_var    = mini_meta_delta.test_var
np_test_var = [np.var(rep1_yes, ddof=1),
               np.var(rep2_yes, ddof=1)]
assert test_var == pytest.approx(np_test_var)

group_var    = mini_meta_delta.group_var
np_group_var = [ci2g.calculate_group_var(control_var[i], N,
                                         test_var[i], N)
                for i in range(0, 2)]
assert group_var == pytest.approx(np_group_var)

test_bootstrap_distribution_variances

In [ ]:
bootstrap_distributions = unpaired.mean_diff.mini_meta.bootstraps
bootstrap_distribution_variances = unpaired.mean_diff.mini_meta.bootstraps_var

np_bootstrap_distribution_variances = np.array([np.var(x, ddof=1) for x in bootstrap_distributions])

assert bootstrap_distribution_variances == pytest.approx(np_bootstrap_distribution_variances)

test_weighted_mean_delta

In [ ]:
difference = unpaired.mean_diff.mini_meta.difference

np_means = np.array([np.mean(rep1_yes)-np.mean(rep1_no), 
            np.mean(rep2_yes)-np.mean(rep2_no)])

np_var   = np_bootstrap_distribution_variances

np_difference = effsize.weighted_delta(np_means, np_var)

assert difference == pytest.approx(np_difference)
weight = np.true_divide(1, np_var)
np.sum(np_means*weight)/np.sum(weight)

-5.819190882223082

test_unpaired_permutation_test

In [ ]:
mini_meta_delta   = unpaired.mean_diff.mini_meta
pvalue             = mini_meta_delta.pvalue_permutation
permutations_delta = mini_meta_delta.permutations_weighted_delta

perm_test_1 = PermutationTest(rep1_no, rep1_yes, 
                            effect_size="mean_diff", 
                            is_paired=False)
perm_test_2 = PermutationTest(rep2_no, rep2_yes, 
                            effect_size="mean_diff", 
                            is_paired=False)
permutations_1 = perm_test_1.permutations
permutations_2 = perm_test_2.permutations
permutations_1_var = perm_test_1.permutations_var
permutations_2_var = perm_test_2.permutations_var

weight_1 = np.true_divide(1,permutations_1_var)
weight_2 = np.true_divide(1,permutations_2_var)

weighted_deltas = (weight_1*permutations_1 + weight_2*permutations_2)/(weight_1+weight_2)
assert permutations_delta == pytest.approx(weighted_deltas)


np_means = [np.mean(rep1_yes)-np.mean(rep1_no), 
            np.mean(rep2_yes)-np.mean(rep2_no)]
np_var   = [np.var(rep1_yes, ddof=1)/N+np.var(rep1_no, ddof=1)/N,
            np.var(rep2_yes, ddof=1)/N+np.var(rep2_no, ddof=1)/N]
np_weight= np.true_divide(1, np_var)

np_difference = np.sum(np_means*np_weight)/np.sum(np_weight)

np_pvalues = len(list(filter(lambda x: np.abs(x)>np.abs(np_difference), 
                            weighted_deltas)))/len(weighted_deltas)

pvalue
# assert pvalue == pytest.approx(np_pvalues)

0.0112